In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
import yaml

sys.path.append(os.path.dirname(os.path.abspath('')))
#from utils.spark import restart_spark
from auto_ab.stratification.params import SplitBuilderParams
from auto_ab.prepilot.params import PrepilotParams
from auto_ab.auto_ab.abtest import ABTest
from auto_ab.auto_ab.params import ABTestParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2
from auto_ab.stratification.split_builder import StratificationSplitBuilder

# Кейс №6. Выделение групп

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [ ]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': 10000,
        'target': 10000,
        'target1': 10000
    },
    region_col = "moda_city",
    split_metric_col = "height_now",
    customer_col = "id",
    cols = [],
    cat_cols=["country"
        
    ],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

In [ ]:
split_builder = StratificationSplitBuilder(df, split_builder_params)

In [ ]:
guests_data_with_strata = split_builder.assign_strata()

In [ ]:
guests_data_with_strata.head()

In [ ]:
groups = split_builder.build_split(guests_data_with_strata)

In [ ]:
groups